In [1]:
import pandas as pd
import numpy as np
import os
from draftSimulator import DraftSimulator
from fantasyTeam import Team
from seasonSimulator import SeasonSimulator
pd.options.mode.chained_assignment = None 


In [2]:
np.random.seed(42) 

script_dir = os.path.dirname(os.path.abspath(''))
data_dir = os.path.join(script_dir, 'data')
data_file_path = os.path.join(data_dir, 'ppr-adp-2023-updated.csv')
weekly_stats_path = os.path.join(data_dir, 'weekly-stats-2022.csv')

myTeam = Team("MahomiBear", 3)
leagueMembers = [
    ('Gridiron Gladiators', 10),
    ('Pigskin Pirates', 2),
    ('Touchdown Titans', 4),
    ('Blitzkrieg Bandits', 1),
    ('End Zone Avengers', 7),
    ('Hail Mary Heroes', 5),
    ('Fumble Force', 9),
    ('Red Zone Renegades', 6),
    ('Sack Masters', 8)
]

draft = DraftSimulator(path=data_file_path, myTeam=myTeam,
                            leagueMembers=leagueMembers, leagueSize=len(leagueMembers) + 1,
                            numRounds=16, model=None, stats=weekly_stats_path)

In [3]:
draftTeams = draft.teams.copy()
for _ in range(draft.numRounds):
    for team in draftTeams:
        # add conditional for 'me' when model created
        response = draft.otherTeamSelection(team)
        if response:
            player_name, playerTeam, position, byeWeek, status, avgadp = response
            team.addPickToRoster(position, player_name, draft.currentPick,
                                    avgadp, playerTeam, byeWeek, 0, status)
        else:
            print('error occured in selecting draft pick')
        draft.currentPick += 1
    draft.currentRound += 1
    draftTeams.reverse()
draft_picks = draft.draftPicksBoard

In [4]:
for team in draft.teams:
    print(team.name)
    print(team.strategy)
    print(team.roster)

Blitzkrieg Bandits
('EarlyRoundQB', 'HeroRB', 'None', 'MidRoundTE', 'MidK', 'MidDST')
   FantasyPosition                 Name Position  PickNumber  \
0               QB           Josh Allen       QB          20   
1              RB1  Christian McCaffrey       RB           1   
2              RB2        Isiah Pacheco       RB          80   
3              WR1        Jaylen Waddle       WR          21   
4              WR2      DeAndre Hopkins       WR          40   
5               TE         Travis Kelce       TE          41   
6             FLEX         Chris Godwin       WR          60   
7                K         Jake Elliott        K         140   
8              DST     Baltimore Ravens      DST         121   
9              BE1          Jerry Jeudy       WR          61   
10             BE2           Mike Evans       WR          81   
11             BE3      Zach Charbonnet       RB         100   
12             BE4        Samaje Perine       RB         101   
13             BE5

In [5]:
waiver_wire_df = draft.constructWaiverWire()
waiver_wire_df.drop(columns=['Rank'], inplace=True)

In [6]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, '..', 'data')
weekly_info_path = os.path.join(data_dir, 'simulator-weekly-info-2023.csv')
season = SeasonSimulator(teams=draft.teams, weekly_info_path=weekly_info_path, waiver_wire_df=waiver_wire_df)

In [7]:
season.matchups

{1: [('Blitzkrieg Bandits', 'Gridiron Gladiators'),
  ('Pigskin Pirates', 'Fumble Force'),
  ('MahomiBear', 'Sack Masters'),
  ('Touchdown Titans', 'End Zone Avengers'),
  ('Hail Mary Heroes', 'Red Zone Renegades')],
 2: [('Blitzkrieg Bandits', 'Fumble Force'),
  ('Gridiron Gladiators', 'Sack Masters'),
  ('Pigskin Pirates', 'End Zone Avengers'),
  ('MahomiBear', 'Red Zone Renegades'),
  ('Touchdown Titans', 'Hail Mary Heroes')],
 3: [('Blitzkrieg Bandits', 'Sack Masters'),
  ('Fumble Force', 'End Zone Avengers'),
  ('Gridiron Gladiators', 'Red Zone Renegades'),
  ('Pigskin Pirates', 'Hail Mary Heroes'),
  ('MahomiBear', 'Touchdown Titans')],
 4: [('Blitzkrieg Bandits', 'End Zone Avengers'),
  ('Sack Masters', 'Red Zone Renegades'),
  ('Fumble Force', 'Hail Mary Heroes'),
  ('Gridiron Gladiators', 'Touchdown Titans'),
  ('Pigskin Pirates', 'MahomiBear')],
 5: [('Blitzkrieg Bandits', 'Red Zone Renegades'),
  ('End Zone Avengers', 'Hail Mary Heroes'),
  ('Sack Masters', 'Touchdown Titans

In [8]:
season.waiverWire.waiver_wire

,Name,Team,ByeWeek,Position,AverageDraftPositionPPR,Status,PointsPerGame,ProjectedFantasyPoints,FantasyPoints
0,Parker Washington,JAX,9,WR,1000.0,INA,0.0,0.0,0.0
1,Elijah Moore,CLE,5,WR,116.0,ACT,0.0,0.0,0.0
2,Adam Thielen,CAR,7,WR,123.0,ACT,0.0,0.0,0.0
3,Allen Lazard,NYJ,7,WR,124.0,ACT,0.0,0.0,0.0
4,Jakobi Meyers,LV,13,WR,130.0,ACT,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
678,Roschon Johnson,CHI,13,RB,162.0,ACT,0.0,0.0,0.0
679,Matthew Stafford,LAR,10,QB,167.0,ACT,0.0,0.0,0.0
680,KJ Osborn,MIN,13,WR,168.0,ACT,0.0,0.0,0.0
681,Kendre Miller,NO,11,RB,169.0,INA,0.0,0.0,0.0


## Week 1

In [9]:
week = 1

In [10]:
# checked attributes of season and looked good
# must now try to simulate a week and see what happens 
season.update_player_status_points(week)

In [11]:
current_waiver_wire = season.waiverWire.waiver_wire
for team in season.teams:
    print(team.name)
    print(team.roster)

Blitzkrieg Bandits
   FantasyPosition                 Name Position  PickNumber  \
0               QB           Josh Allen       QB          20   
1              RB1  Christian McCaffrey       RB           1   
2              RB2        Isiah Pacheco       RB          80   
3              WR1        Jaylen Waddle       WR          21   
4              WR2      DeAndre Hopkins       WR          40   
5               TE         Travis Kelce       TE          41   
6             FLEX         Chris Godwin       WR          60   
7                K         Jake Elliott        K         140   
8              DST     Baltimore Ravens      DST         121   
9              BE1          Jerry Jeudy       WR          61   
10             BE2           Mike Evans       WR          81   
11             BE3      Zach Charbonnet       RB         100   
12             BE4        Samaje Perine       RB         101   
13             BE5       Tyler Allgeier       RB         120   
14             BE6   

In [12]:
season.update_rosters()

Team(Gridiron Gladiators)
K streaming: True
DST streaming: True
Waiver wire status: 0
roster status: 1
[(None,           Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Greg Joseph  MIN       13        K                    272.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                    8.12            0.0  ), (None,                Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Seattle Seahawks  SEA        5      DST                    231.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                     8.6            0.0  )]
Team(Fumble Force)
K streaming: True
DST streaming: False
Waiver wire status: 0
roster status: 1
[(None,           Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Greg Joseph  MIN       13        K                    272.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                

In [19]:
season.team_dict['Hail Mary Heroes'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Joe Burrow,QB,45,41.0,CIN,7,0.0,ACT,18.99,3.18
1,RB1,Bijan Robinson,RB,5,8.0,ATL,11,0.0,ACT,15.56,20.30
2,RB2,Alexander Mattison,RB,56,56.0,MIN,13,0.0,ACT,13.21,13.40
3,WR1,Garrett Wilson,WR,16,17.0,NYJ,7,0.0,ACT,14.74,14.40
4,WR2,Tyler Lockett,WR,65,65.0,SEA,5,0.0,ACT,13.57,3.00
5,TE,David Njoku,TE,96,98.0,CLE,5,0.0,ACT,9.81,4.40
6,FLEX,Tee Higgins,WR,25,29.0,CIN,7,0.0,ACT,13.55,0.00
7,K,Greg Joseph,K,0,272.0,MIN,13,0.0,ACT,8.12,5.00
8,DST,Denver Broncos,DST,0,203.0,DEN,9,0.0,ACT,8.30,3.00
9,BE1,Amari Cooper,WR,36,39.0,CLE,5,0.0,ACT,13.15,6.70


In [14]:
for team in season.teams:
    print(team)
    print(team.roster[team.roster['Position'] == 'DST'][['Name', 'ProjectedFantasyPoints']])
    print(team.roster[team.roster['Position'] == 'K'][['Name', 'ProjectedFantasyPoints']])


Team(Blitzkrieg Bandits)
               Name  ProjectedFantasyPoints
8  Baltimore Ravens                     8.5
              Name  ProjectedFantasyPoints
7  Riley Patterson                    7.78
Team(Pigskin Pirates)
                Name  ProjectedFantasyPoints
8  Green Bay Packers                     8.2
         Name  ProjectedFantasyPoints
7  Tyler Bass                    7.98
Team(MahomiBear)
                  Name  ProjectedFantasyPoints
8  Philadelphia Eagles                     8.2
            Name  ProjectedFantasyPoints
7  Jason Sanders                    7.88
Team(Touchdown Titans)
                   Name  ProjectedFantasyPoints
8  New England Patriots                     5.7
             Name  ProjectedFantasyPoints
7  Evan McPherson                    8.22
Team(Hail Mary Heroes)
             Name  ProjectedFantasyPoints
8  Denver Broncos                     8.3
          Name  ProjectedFantasyPoints
7  Greg Joseph                    8.12
Team(Red Zone Renegades)
       

In [15]:
season.waiverWire.waiver_wire[season.waiverWire.waiver_wire['Position'] == 'DST']

,Name,Team,ByeWeek,Position,AverageDraftPositionPPR,Status,PointsPerGame,ProjectedFantasyPoints,FantasyPoints
46,Cleveland Browns,CLE,5,DST,233.0,ACT,0.0,6.4,0.0
60,Cincinnati Bengals,CIN,7,DST,214.0,ACT,0.0,6.0,0.0
82,Los Angeles Rams,LAR,10,DST,269.0,ACT,0.0,5.8,0.0
88,Carolina Panthers,CAR,7,DST,259.0,ACT,0.0,5.2,0.0
92,New York Giants,NYG,13,DST,247.0,ACT,0.0,5.3,0.0
95,Minnesota Vikings,MIN,13,DST,250.0,ACT,0.0,7.2,0.0
96,Los Angeles Chargers,LAC,5,DST,251.0,ACT,0.0,5.8,0.0
98,Detroit Lions,DET,9,DST,253.0,ACT,0.0,5.1,0.0
108,Indianapolis Colts,IND,11,DST,297.0,ACT,0.0,5.1,0.0
110,Tennessee Titans,TEN,7,DST,300.0,ACT,0.0,5.7,0.0


In [16]:
season.waiverWire.waiver_wire[season.waiverWire.waiver_wire['Position'] == 'K']

,Name,Team,ByeWeek,Position,AverageDraftPositionPPR,Status,PointsPerGame,ProjectedFantasyPoints,FantasyPoints
51,Graham Gano,NYG,13,K,222.0,ACT,0.0,7.62,0.0
56,Greg Zuerlein,NYJ,7,K,227.0,ACT,0.0,6.96,0.0
61,Jake Moody,SF,9,K,215.0,ACT,0.0,6.84,0.0
63,Matt Gay,IND,11,K,217.0,ACT,0.0,7.28,0.0
72,Cameron Dicker,LAC,5,K,209.0,INA,0.0,0.00,0.0
130,Brett Maher,LAR,10,K,287.0,ACT,0.0,6.54,0.0
155,Jake Verity,TB,5,K,313.0,INA,0.0,0.00,0.0
162,Michael Badgley,DET,9,K,320.0,INA,0.0,0.00,0.0
163,Cade York,TEN,7,K,305.0,INA,0.0,0.00,0.0
180,Matt Prater,ARI,14,K,361.0,ACT,0.0,6.14,0.0


In [17]:
season.waiverWire.waiver_wire

,Name,Team,ByeWeek,Position,AverageDraftPositionPPR,Status,PointsPerGame,ProjectedFantasyPoints,FantasyPoints
0,Parker Washington,JAX,9,WR,1000.0,INA,0.0,0.00,0.0
1,Elijah Moore,CLE,5,WR,116.0,ACT,0.0,9.68,0.0
2,Adam Thielen,CAR,7,WR,123.0,ACT,0.0,9.01,0.0
3,Allen Lazard,NYJ,7,WR,124.0,ACT,0.0,9.08,0.0
4,Jakobi Meyers,LV,13,WR,130.0,ACT,0.0,9.13,0.0
...,...,...,...,...,...,...,...,...,...
679,Jake Elliott,PHI,10,K,185.0,ACT,0.0,7.68,0.0
680,Pittsburgh Steelers,PIT,6,DST,164.0,ACT,0.0,6.30,0.0
681,Brandon McManus,JAX,9,K,195.0,ACT,0.0,7.78,0.0
682,Kansas City Chiefs,KC,10,DST,175.0,ACT,0.0,5.70,0.0


In [18]:
season.simulate_week(week)

Week 1 - Blitzkrieg Bandits vs Gridiron Gladiators: 100.84 - 89.42
Week 1 - Pigskin Pirates vs Fumble Force: 116.74 - 112.04
Week 1 - MahomiBear vs Sack Masters: 109.46000000000001 - 118.96000000000001
Week 1 - Touchdown Titans vs End Zone Avengers: 72.8 - 112.86000000000001
Week 1 - Hail Mary Heroes vs Red Zone Renegades: 66.68 - 159.24


In [20]:
season.standings

,Team,Wins,Losses,Points For,Points Against
5,Red Zone Renegades,1,0,159.24,66.68
7,Sack Masters,1,0,118.96,109.46
1,Pigskin Pirates,1,0,116.74,112.04
6,End Zone Avengers,1,0,112.86,72.80
0,Blitzkrieg Bandits,1,0,100.84,89.42
8,Fumble Force,0,1,112.04,116.74
2,MahomiBear,0,1,109.46,118.96
9,Gridiron Gladiators,0,1,89.42,100.84
3,Touchdown Titans,0,1,72.80,112.86
4,Hail Mary Heroes,0,1,66.68,159.24


In [21]:
season.team_dict['Touchdown Titans'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Jalen Hurts,QB,24,24.0,PHI,10,0.0,ACT,22.28,12.50
1,RB1,Derrick Henry,RB,17,18.0,TEN,7,0.0,ACT,16.47,13.90
2,RB2,Raheem Mostert,RB,144,134.0,MIA,10,0.0,ACT,12.48,13.00
3,WR1,Ja'Marr Chase,WR,4,3.0,CIN,7,0.0,ACT,18.61,9.10
4,WR2,Drake London,WR,57,57.0,ATL,11,0.0,ACT,12.32,0.00
5,TE,Darren Waller,TE,64,60.0,NYG,13,0.0,ACT,10.61,6.60
6,FLEX,Marquise Brown,WR,84,81.0,ARI,14,0.0,ACT,11.15,8.70
7,K,Evan McPherson,K,137,183.0,CIN,7,0.0,ACT,8.22,3.00
8,DST,New England Patriots,DST,117,139.0,NE,11,0.0,ACT,5.70,6.00
9,BE1,D'Andre Swift,RB,77,74.0,PHI,10,0.0,ACT,10.46,1.30


In [20]:
season._update_points_per_game(week)

Week 1 went as expected

## Week 2

In [22]:
# update weeks for each team, waiverwire, and variable
week += 1
for team in season.teams:
    team.currentWeek = week
    team.positionsInNeed = []
    team.goingToDrop = []
    team.goingToAdd = []
season.waiverWire.week = week


In [23]:
season.update_player_status_points(week)

In [24]:
season.update_rosters()

Team(Gridiron Gladiators)
K streaming: True
DST streaming: True
Waiver wire status: 0
roster status: 1
[(None,            Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Jake Elliott  PHI       10        K                    185.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                    8.74            0.0  ), (None,                   Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Pittsburgh Steelers  PIT        6      DST                    164.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                     8.7            0.0  )]
Team(Fumble Force)
K streaming: True
DST streaming: False
Waiver wire status: 0
roster status: 1
[(None,               Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Harrison Butker   KC       10        K                    160.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0

In [25]:
for team in season.teams:
    print(team)
    print(team.roster[team.roster['Position'] == 'DST'][['Name', 'ProjectedFantasyPoints']])
    print(team.roster[team.roster['Position'] == 'K'][['Name', 'ProjectedFantasyPoints']])

Team(Blitzkrieg Bandits)
             Name  ProjectedFantasyPoints
8  Houston Texans                     7.9
              Name  ProjectedFantasyPoints
7  Riley Patterson                    8.18
Team(Pigskin Pirates)
                    Name  ProjectedFantasyPoints
8  Washington Commanders                     8.0
         Name  ProjectedFantasyPoints
7  Tyler Bass                    8.38
Team(MahomiBear)
                  Name  ProjectedFantasyPoints
8  Philadelphia Eagles                     8.0
             Name  ProjectedFantasyPoints
7  Evan McPherson                    8.12
Team(Touchdown Titans)
                   Name  ProjectedFantasyPoints
8  New England Patriots                     6.5
         Name  ProjectedFantasyPoints
7  Jake Moody                    8.22
Team(Hail Mary Heroes)
             Name  ProjectedFantasyPoints
8  Denver Broncos                     8.0
           Name  ProjectedFantasyPoints
7  Younghoe Koo                    8.24
Team(Red Zone Renegades)
       

In [26]:
season.waiverWire.waiver_wire[season.waiverWire.waiver_wire['Position'] == 'DST']

,Name,Team,ByeWeek,Position,AverageDraftPositionPPR,Status,PointsPerGame,ProjectedFantasyPoints,FantasyPoints
46,Cleveland Browns,CLE,5,DST,233.0,ACT,0.0,7.8,0.0
60,Cincinnati Bengals,CIN,7,DST,214.0,ACT,0.0,7.5,0.0
82,Los Angeles Rams,LAR,10,DST,269.0,ACT,0.0,4.9,0.0
88,Carolina Panthers,CAR,7,DST,259.0,ACT,0.0,7.0,0.0
92,New York Giants,NYG,13,DST,247.0,ACT,0.0,7.7,0.0
95,Minnesota Vikings,MIN,13,DST,250.0,ACT,0.0,5.1,0.0
96,Los Angeles Chargers,LAC,5,DST,251.0,ACT,0.0,6.6,0.0
98,Detroit Lions,DET,9,DST,253.0,ACT,0.0,6.0,0.0
108,Indianapolis Colts,IND,11,DST,297.0,ACT,0.0,6.6,0.0
110,Tennessee Titans,TEN,7,DST,300.0,ACT,0.0,5.3,0.0


In [27]:
season.simulate_week(week)

Week 2 - Blitzkrieg Bandits vs Fumble Force: 103.96 - 91.84
Week 2 - Gridiron Gladiators vs Sack Masters: 139.8 - 129.5
Week 2 - Pigskin Pirates vs End Zone Avengers: 117.89999999999999 - 116.30000000000001
Week 2 - MahomiBear vs Red Zone Renegades: 134.98000000000002 - 118.64
Week 2 - Touchdown Titans vs Hail Mary Heroes: 121.52 - 126.78


In [28]:
season._update_points_per_game(week)

## Week 3

In [29]:
week += 1
for team in season.teams:
    team.currentWeek = week
    team.positionsInNeed = []
    team.goingToDrop = []
    team.goingToAdd = []
season.waiverWire.week = week

In [30]:
season.update_player_status_points(week)

In [31]:
season.update_rosters()

Team(Gridiron Gladiators)
K streaming: True
DST streaming: True
Waiver wire status: 0
roster status: 1
[(None,               Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Brandon McManus  JAX        9        K                    195.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                    8.54            0.0  ), (None,                  Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Kansas City Chiefs   KC       10      DST                    175.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0                     9.4            0.0  )]
Team(Fumble Force)
K streaming: True
DST streaming: False
Waiver wire status: 0
roster status: 1
[(None,             Name Team  ByeWeek Position  AverageDraftPositionPPR Status  \
0  Jason Sanders  MIA       10        K                    220.0    ACT   

   PointsPerGame  ProjectedFantasyPoints  FantasyPoints  
0            0.0

In [32]:
season.waiverWire.waiver_wire

,Name,Team,ByeWeek,Position,AverageDraftPositionPPR,Status,PointsPerGame,ProjectedFantasyPoints,FantasyPoints
0,Parker Washington,JAX,9,WR,1000.0,INA,0.0,0.00,0.0
1,Elijah Moore,CLE,5,WR,116.0,ACT,0.0,8.80,0.0
2,Adam Thielen,CAR,7,WR,123.0,ACT,0.0,9.55,0.0
3,Allen Lazard,NYJ,7,WR,124.0,ACT,0.0,7.87,0.0
4,Jakobi Meyers,LV,13,WR,130.0,ACT,0.0,10.99,0.0
...,...,...,...,...,...,...,...,...,...
665,Washington Commanders,WAS,14,DST,200.0,ACT,0.0,6.30,10.0
668,Tampa Bay Buccaneers,TB,5,DST,295.0,ACT,0.0,6.30,17.0
669,Pittsburgh Steelers,PIT,6,DST,164.0,ACT,0.0,6.80,26.0
667,Denver Broncos,DEN,9,DST,203.0,ACT,0.0,5.30,0.0


In [33]:
for team in season.team_dict:
    print(team)
    print(season.team_dict[team].roster)

Blitzkrieg Bandits
   FantasyPosition                 Name Position  PickNumber  \
0               QB           Josh Allen       QB          20   
1              RB1  Christian McCaffrey       RB           1   
2              RB2        Isiah Pacheco       RB          80   
3              WR1           Mike Evans       WR          81   
4              WR2         Chris Godwin       WR          60   
5               TE         Travis Kelce       TE          41   
6             FLEX      DeAndre Hopkins       WR          40   
7                K      Riley Patterson        K           0   
8              DST   Cincinnati Bengals      DST           0   
9              BE1          Jerry Jeudy       WR          61   
10             BE2        Jaylen Waddle       WR          21   
11             BE3      Zach Charbonnet       RB         100   
12             BE4        Samaje Perine       RB         101   
13             BE5       Tyler Allgeier       RB         120   
14             BE6   

In [34]:
season.simulate_week(week)

Week 3 - Blitzkrieg Bandits vs Sack Masters: 131.92000000000002 - 138.20000000000002
Week 3 - Fumble Force vs End Zone Avengers: 138.26000000000002 - 140.22
Week 3 - Gridiron Gladiators vs Red Zone Renegades: 121.06 - 129.26
Week 3 - Pigskin Pirates vs Hail Mary Heroes: 118.67999999999999 - 102.28
Week 3 - MahomiBear vs Touchdown Titans: 159.68 - 137.07999999999998


In [35]:
season.team_dict['Fumble Force'].roster

,FantasyPosition,Name,Position,PickNumber,AverageDraftPositionPPR,Team,ByeWeek,PointsPerGame,Status,ProjectedFantasyPoints,FantasyPoints
0,QB,Justin Fields,QB,49,46.0,CHI,13,0.0,ACT,16.80,10.66
1,RB1,Joe Mixon,RB,32,32.0,CIN,7,0.0,ACT,15.71,14.00
2,RB2,Devin Singletary,RB,149,147.0,HOU,7,0.0,ACT,0.00,6.50
3,WR1,Stefon Diggs,WR,9,10.0,BUF,13,0.0,ACT,17.88,19.10
4,WR2,DK Metcalf,WR,29,31.0,SEA,5,0.0,ACT,13.37,17.20
5,TE,Kyle Pitts,TE,72,73.0,ATL,11,0.0,ACT,8.60,8.70
6,FLEX,Courtland Sutton,WR,89,90.0,DEN,9,0.0,ACT,9.78,19.10
7,K,Harrison Butker,K,0,160.0,KC,10,0.0,ACT,9.14,11.00
8,DST,Buffalo Bills,DST,112,128.0,BUF,13,0.0,ACT,8.30,32.00
9,BE1,Alvin Kamara,RB,69,69.0,NO,11,0.0,RES,0.00,0.00


In [36]:
season._update_points_per_game(week)

## Week 4

In [ ]:
for team in season.teams:
    team.currentWeek = week
    team.positionsInNeed = []
    team.goingToDrop = []
    team.goingToAdd = []
season.waiverWire.week = week
season.update_player_status_points(week)
self.update_rosters()
self.simulate_week(week)
self._update_points_per_game(week)